In [10]:
pip install mlagents==0.17.0
pip install mlagents-env==0.17.0
pip install gym_unity==0.17.0

SyntaxError: invalid syntax (642488665.py, line 1)

In [20]:
cd

/Users/dohatamayu


In [29]:
ls

CODE_OF_CONDUCT.md                markdown-link-check.fast.json
DevProject/                       markdown-link-check.full.json
Dockerfile                        ml-agents/
LICENSE.md                        ml-agents-envs/
Project/                          ml-agents-plugin-examples/
README.md                         protobuf-definitions/
SURVEY.md                         pytest.ini
colab/                            setup.cfg
colab_requirements.txt            test_constraints_max_version.txt
com.unity.ml-agents/              test_constraints_mid_version.txt
com.unity.ml-agents.extensions/   test_constraints_min_version.txt
config/                           test_requirements.txt
docs/                             unity-volume/
gym-unity/                        utils/


In [23]:
cd Downloads/

/Users/dohatamayu/Downloads


In [25]:
cd ml-agents/

/Users/dohatamayu/Downloads/ml-agents


In [28]:
cd ../ml-agents-release_19/

/Users/dohatamayu/Downloads/ml-agents-release_19


In [35]:
import gym

env=gym.make("CartPole-v0")

for episode in range(20):
    env.reset()
    for t in range(100):
        action=env.action_space.sample()
        env.step(action)
        env.render()

/Users/dohatamayu/opt/anaconda3/lib/python3.9/site-packages/gym/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [106]:
import gym  #倒立振子(cartpole)の実行環境
from gym import wrappers
from gym.wrappers.monitoring.video_recorder import VideoRecorder #ビデオ保存(なぜか出来ない)
import numpy as np
import time
 
# 状態の４変数→離散値にデジタル変換する関数
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]
 
# 各値を離散値に変換
def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [
        np.digitize(cart_pos, bins=bins(-4.8, 4.8, num_dizitized)),
        np.digitize(cart_v, bins=bins(-3.0, 3.0, num_dizitized)),
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, num_dizitized)),
        np.digitize(pole_v, bins=bins(-2.0, 2.0, num_dizitized))
    ]
    return sum([x * (num_dizitized**i) for i, x in enumerate(digitized)])
  
# 行動a(t)を求める関数
def get_action(next_state, episode):
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[next_state])
    else:
        next_action = np.random.choice([0, 1])
    return next_action
 

# Qテーブルを更新する関数
def update_Qtable(q_table, state, action, reward, next_state):
    gamma = 0.99
    alpha = 0.5
    next_Max_Q=max(q_table[next_state][0],q_table[next_state][1] )
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * next_Max_Q)
   
    return q_table
 
#パラメーター設定
env = gym.make('CartPole-v1')
max_number_of_steps = 200  #1試行のstep数
num_consecutive_iterations = 100  #学習完了評価に使用する平均試行回数
num_episodes = 1000  #総試行回数
goal_average_reward = 195  #この報酬を超えると学習終了（中心への制御なし）

# 状態を6^4変数にデジタル変換してQ関数（表）を作成
num_dizitized = 6  #分割数
q_table = np.random.uniform(
    low=-1, high=1, size=(num_dizitized**4, env.action_space.n))
 
total_reward_vec = np.zeros(num_consecutive_iterations)  #各試行の報酬を格納
final_x = np.zeros((num_episodes, 1))  #学習後、各試行のt=200でのｘの位置を格納
islearned = 0  #学習が終わったフラグ
isrender = 0  #描画フラグ
 
for episode in range(num_episodes):  #試行数分繰り返す
    # 環境の初期化
    observation = env.reset()
    print(observation)
    state = digitize_state(observation[0])
    action = np.argmax(q_table[state])
    episode_reward = 0
 
    for t in range(max_number_of_steps):  #1試行のループ
        if islearned == 1:  #学習終了したらcartPoleを描画する
            env.render()
            time.sleep(0.1)
            print (observation[0])  #カートのx位置を出力
 
        # 行動a(t)の実行により、s(t+1), r(t)などを計算する
        observation, reward, done, info, buffer= env.step(action)
 
        # 報酬を設定し与える
        if done:
            if t < 195:
                reward = -200  #こけたら罰則
            else:
                reward = 1  #立ったまま終了時は罰則はなし
        else:
            reward = 1  #各ステップで立ってたら報酬追加
 
        episode_reward += reward  #報酬を追加
 
        # 離散状態s(t+1)を求め、Q関数を更新する
        next_state = digitize_state(observation)  #t+1での観測状態を、離散値に変換
        q_table = update_Qtable(q_table, state, action, reward, next_state)
        
        #  次の行動a(t+1)を求める 
        action = get_action(next_state, episode)   
        
        state = next_state
        
        #終了時の処理
        if done:
            print('%d Episode finished after %f time steps / mean %f' %
                  (episode, t + 1, total_reward_vec.mean()))
            total_reward_vec = np.hstack((total_reward_vec[1:],
                                          episode_reward))  #報酬を記録
            if islearned == 1:  #学習終わってたら最終のx座標を格納
                final_x[episode, 0] = observation[0]
            break
        
    if (total_reward_vec.mean() >=
        goal_average_reward):  # 直近の100エピソードが規定報酬以上であれば成功
    print('Episode %d train agent successfuly!' % episode)
    islearned = 1
    if isrender == 0:
        env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
        isrender = 1
 
if islearned:
    np.savetxt('final_x.csv', final_x, delimiter=",")

(array([-0.04179334,  0.01448609,  0.03180197, -0.01423315], dtype=float32), {})
(array([ 0.02842028, -0.04239586, -0.00871196,  0.01066262], dtype=float32), {})
(array([-0.01548824, -0.02466642, -0.00604543,  0.04782628], dtype=float32), {})
(array([-0.03150218,  0.04131653, -0.04220275, -0.04362871], dtype=float32), {})
(array([ 0.02243142,  0.01180685, -0.03065548, -0.02843328], dtype=float32), {})
(array([-0.02012167,  0.04774988, -0.01382863, -0.0437659 ], dtype=float32), {})
(array([-0.03150181,  0.03581633,  0.02754766, -0.04347371], dtype=float32), {})
(array([ 7.3400239e-05,  4.2048555e-02,  2.5904264e-02, -2.9042626e-02],
      dtype=float32), {})
(array([ 0.01529112, -0.02745338,  0.0399649 ,  0.00484288], dtype=float32), {})
(array([ 0.01717116, -0.04304041,  0.00639854, -0.03352667], dtype=float32), {})
(array([-0.00839041, -0.02867185, -0.00426229, -0.01638411], dtype=float32), {})
(array([ 0.0470116 ,  0.03087162, -0.04584018, -0.04462966], dtype=float32), {})
(array([-0

(array([-0.01054557, -0.0081825 ,  0.04004688,  0.00624262], dtype=float32), {})
(array([ 0.03808444, -0.0035402 ,  0.0320928 ,  0.02184917], dtype=float32), {})
(array([ 0.00892815,  0.03387323,  0.03906808, -0.00160618], dtype=float32), {})
(array([ 0.01707667, -0.03677878, -0.03182124,  0.02674196], dtype=float32), {})
(array([-0.01159837, -0.00552887,  0.03024245,  0.01896051], dtype=float32), {})
(array([-0.00047055,  0.00316271, -0.01906106, -0.00452687], dtype=float32), {})
(array([ 0.03126995,  0.01079173, -0.04966275, -0.04307729], dtype=float32), {})
(array([-0.04234302, -0.0410669 ,  0.04593462,  0.01834634], dtype=float32), {})
(array([ 0.02076095,  0.02956687, -0.04152864,  0.0146058 ], dtype=float32), {})
184 Episode finished after 14.000000 time steps / mean -56.000000
(array([ 0.03524009,  0.02269047, -0.0138797 , -0.04199041], dtype=float32), {})
185 Episode finished after 15.000000 time steps / mean -74.700000
(array([ 0.01490001, -0.03131192,  0.01357075, -0.01889157

(array([ 0.03779113, -0.02004783,  0.00949059, -0.006034  ], dtype=float32), {})
(array([ 0.00870959, -0.02062972,  0.03944139, -0.02090398], dtype=float32), {})
(array([-0.0443998 , -0.03310711,  0.01587351, -0.0031409 ], dtype=float32), {})
(array([ 0.01322042,  0.04728796,  0.0007608 , -0.03740953], dtype=float32), {})
(array([-0.00235527, -0.01810106,  0.01089484,  0.02676262], dtype=float32), {})
(array([ 0.0057168 ,  0.03199878, -0.04254496,  0.02444589], dtype=float32), {})
(array([-0.00970133,  0.01359734, -0.00348176,  0.02774283], dtype=float32), {})
(array([ 0.01354022, -0.01039346,  0.04155514, -0.02278769], dtype=float32), {})
(array([ 0.04616043,  0.00396373, -0.01123631, -0.0385544 ], dtype=float32), {})
(array([ 0.01390565, -0.0499256 , -0.04264969,  0.04301666], dtype=float32), {})
(array([-0.01878916, -0.02695964, -0.03906736, -0.04296034], dtype=float32), {})
(array([ 0.04394634,  0.00517466, -0.04795105,  0.04275382], dtype=float32), {})
(array([ 0.01697894,  0.0024

(array([-0.03261586, -0.04177016, -0.03791909,  0.00628646], dtype=float32), {})
(array([ 0.04421568, -0.01104887,  0.03505641, -0.01866075], dtype=float32), {})
(array([ 0.03524209,  0.0362373 , -0.01875215, -0.02742046], dtype=float32), {})
(array([ 0.01587926, -0.00640755, -0.04619717, -0.02531634], dtype=float32), {})
(array([-0.03786908, -0.02168549,  0.02309629,  0.03153336], dtype=float32), {})
(array([-0.02531742,  0.04422532, -0.04471358, -0.04456526], dtype=float32), {})
(array([-0.01627065, -0.01501047,  0.04672262,  0.02165434], dtype=float32), {})
(array([-0.0468691 , -0.02791917,  0.04682643,  0.04400498], dtype=float32), {})
(array([-0.04360348, -0.02774537,  0.03387864,  0.04199651], dtype=float32), {})
(array([ 0.02841048,  0.00228253, -0.02896135,  0.04806124], dtype=float32), {})
(array([-0.0067307 ,  0.01194529,  0.03899625, -0.03153175], dtype=float32), {})
(array([-0.04124341,  0.0469848 ,  0.01324324,  0.03022199], dtype=float32), {})
(array([ 0.02356808,  0.0157

(array([-0.00799002,  0.00252464, -0.03658943,  0.00674961], dtype=float32), {})
(array([-0.03024299,  0.02168941, -0.02133276, -0.04084371], dtype=float32), {})
(array([-0.02383737,  0.0023898 , -0.03465112,  0.02951931], dtype=float32), {})
(array([ 0.03260094, -0.00952297, -0.04421546, -0.01928431], dtype=float32), {})
(array([-0.04546914, -0.02941782, -0.04039693, -0.0007244 ], dtype=float32), {})
(array([ 0.01169123, -0.00815387,  0.02436562,  0.01900462], dtype=float32), {})
(array([-0.02455593,  0.04615656, -0.01269228, -0.02432381], dtype=float32), {})
(array([0.04140568, 0.0146478 , 0.01139294, 0.00655495], dtype=float32), {})
(array([ 0.01232615, -0.02625505,  0.03546432, -0.02821816], dtype=float32), {})
(array([-0.0404718 , -0.01001791, -0.02841956,  0.00874018], dtype=float32), {})
(array([-0.0013909 , -0.03839043, -0.03387944, -0.04988192], dtype=float32), {})
(array([ 0.01414684, -0.02170519,  0.02241884,  0.02795335], dtype=float32), {})
(array([0.00127167, 0.00846213, 

(array([ 0.00471342,  0.00333446,  0.04646902, -0.01330281], dtype=float32), {})
(array([-0.012248  , -0.00526152,  0.03846839, -0.02892551], dtype=float32), {})
(array([ 0.00268904, -0.01655294, -0.04779708,  0.01504716], dtype=float32), {})
(array([0.03237001, 0.01871772, 0.03311177, 0.03720831], dtype=float32), {})
(array([-0.0428307 ,  0.02569744, -0.00785472, -0.0357907 ], dtype=float32), {})
(array([0.03537868, 0.03994701, 0.02426212, 0.04616261], dtype=float32), {})
(array([-0.00893019, -0.00158243,  0.02342438,  0.02346789], dtype=float32), {})
(array([ 0.00228413, -0.00573919, -0.00158381, -0.00697493], dtype=float32), {})
(array([-0.02257031, -0.02309212,  0.00871906, -0.04108618], dtype=float32), {})
(array([ 0.00046668, -0.04547554,  0.00765193, -0.03738394], dtype=float32), {})
(array([-0.01910636,  0.02906531, -0.04776809,  0.01933042], dtype=float32), {})
(array([ 0.01037632, -0.04938179,  0.04589572,  0.02155002], dtype=float32), {})
(array([ 0.02094485, -0.02968627,  0

In [82]:
pip install gym==0.15.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.4 MB/s eta 0:00:0000:01
  Created wheel for gym: filename=gym-0.15.3-py3-none-any.whl size=1644943 sha256=c84fd36428d3bd758880980d25ce794ed0bf214f78e8b7283af055c5bf077884
  Stored in directory: /Users/dohatamayu/Library/Caches/pip/wheels/95/4d/2a/1857eeb0a8d603d2a17d9b1f3551908c743db62bca96e44428
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.0.0
    Uninstalling cloudpickle-2.0.0:
      Successfully uninstalled cloudpickle-2.0.0
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [89]:
pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 15.5 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110728 sha256=590d0ee387dc78bde572eb52c2fd2c4b2040eb9091d58e70456630daed054ea7
  Stored in directory: /Users/dohatamayu/Library/Caches/pip/wheels/29/15/e4/4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
Note: you may need to restart the kernel to use updated packages.


In [67]:
type(tuple(observation[0]))

tuple

In [74]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, add, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

(array([-0.01863588,  0.00915914, -0.02434438, -0.04135956], dtype=float32), {})
[-0.01845269 -0.1856054  -0.02517157  0.24354416]
[-0.0221648  -0.38035893 -0.02030069  0.52818227]
[-0.02977198 -0.5751895  -0.00973705  0.81439984]
[-0.04127577 -0.77017677  0.00655095  1.1040043 ]
[-0.0566793  -0.96538424  0.02863104  1.3987352 ]
[-0.07598699 -0.7706297   0.05660574  1.1151395 ]
[-0.09139959 -0.5762947   0.07890853  0.84073704]
[-0.10292548 -0.77240014  0.09572327  1.1571549 ]
[-0.11837348 -0.9686305   0.11886637  1.4782526 ]
[-0.1377461  -1.1649863   0.14843142  1.8055725 ]
[-0.16104582 -0.9718007   0.18454286  1.5624592 ]
Episode finished after 12 timesteps
(array([ 0.01573902,  0.00681485, -0.04549064,  0.03474437], dtype=float32), {})
[ 0.01587532 -0.18762623 -0.04479575  0.31273463]
[ 0.01212279  0.0081043  -0.03854106  0.0062678 ]
[ 0.01228488  0.20375718 -0.0384157  -0.29832184]
[ 0.01636003  0.0092033  -0.04438214 -0.01799776]
[ 0.01654409 -0.18525499 -0.04474209  0.26035836]
[ 